In [0]:
# %pip install mlflow

In [0]:
# %pip install --upgrade typing_extensions statsmodels

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('/Volumes/training_2025/mayank/time_series_prac/bitcoin_price.csv')
df.head(5)

In [0]:
import pandas as pd
import mlflow
import mlflow.pyfunc
from statsmodels.tsa.arima.model import ARIMA
import pickle
import os
from mlflow.models import infer_signature


class ARIMAModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        with open(context.artifacts["arima_model"], "rb") as f:
            self.model = pickle.load(f)

    def predict(self, context, model_input):
        steps = len(model_input)
        return self.model.forecast(steps=steps)

def main():
    mlflow.set_experiment("/Workspace/Users/mayank.bhadauria@datanimbus.com/ml_asset_model/btc_arima_test")

    df = pd.read_csv("/Volumes/training_2025/mayank/time_series_prac/bitcoin_price.csv")
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date")

    close_series = df["Close"]

    # Train
    model = ARIMA(close_series, order=(5,1,0))
    fitted = model.fit()

    example_input = pd.DataFrame({"dummy": [0.0] * 7})
    example_output = fitted.forecast(steps=len(example_input))

    signature = infer_signature(example_input, example_output)

    # Save model locally first
    os.makedirs("models", exist_ok=True)
    model_path = "models/arima_model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(fitted, f)

    with mlflow.start_run():
        mlflow.log_param("order", (5,1,0))
        mlflow.log_metric("final_price", close_series.iloc[-1])

        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=ARIMAModelWrapper(),
            artifacts={"arima_model": model_path},
            registered_model_name="training_2025.mayank.btc_arima_model",
            signature=signature
        )

    print("Model successfully logged.")

if __name__ == "__main__":
    main()


In [0]:
import mlflow
import pandas as pd

def main():
    model_uri = "models:/training_2025.mayank.btc_arima_model/4" 
    model = mlflow.pyfunc.load_model(model_uri)

    future_steps = 7
    input_df = pd.DataFrame({"dummy": [0.0] * future_steps})

    preds = model.predict(input_df)
    print("7-step forecast:")
    print(preds)

if __name__ == "__main__":
    main()
